### Multi-criteria optimization

In [ ]:
using CSV
using DataFrames
using Statistics
using JuMP
using MultiJuMP
using Ipopt
using Plots

prices = DataFrame(CSV.File("10_stocks_3yr.csv"))
prices[1:5,:]


In [ ]:
prices.rateOfRet = (prices.close .- prices.open) ./ prices.open
dats = unstack(prices, :date, :Name, :rateOfRet)
dats


In [ ]:
const avg_rets = [ mean(col) for col in eachcol(dats[:,2:end])]
const cov_mx = cov(Matrix(dats[:,2:end]))


In [ ]:
m = multi_model(Ipopt.Optimizer)
@variable(m, 0 <= x[i=1:10] <= 1)
@constraint(m,sum(x) == 1)

@variable(m, risk)
@constraint(m, risk == x'*cov_mx*x)

@variable(m, rets)
@constraint(m, rets == avg_rets' * x)

@NLexpression(m, f_risk, risk)
@NLexpression(m, f_rets, rets)

iv1 =  fill(0.1, 10) # Initial guess
obj1 = SingleObjective(f_risk, sense = MOI.MIN_SENSE,
                        iv = Dict{String,Any}("x[$i]" => iv1[i] for i in 1:length(iv1)))
obj2 = SingleObjective(f_rets, sense = MOI.MAX_SENSE)

md = get_multidata(m)
md.objectives = [obj1, obj2]
md.pointsperdim = 20;


In [ ]:
optimize!(m, method = NBI(false))

In [ ]:
pyplot()
pltnbi = Plots.plot(md)
